In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
model = AutoModelForSequenceClassification.from_pretrained("Valleyy/vntc-based-text-classification", ignore_mismatched_sizes=True)

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

In [3]:
import torch
model.eval()
# Di chuyển model lên GPU
model.to('cuda')

model_config = model.config
labels = model_config.id2label

# Function to classify text
def classify_text(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", max_length=256, truncation=True, padding=True)

    # Di chuyển dữ liệu lên GPU
    inputs = {key: tensor.to('cuda') for key, tensor in inputs.items()}

    # Forward pass, get logits
    with torch.no_grad():
        outputs = model(**inputs)

    # Convert logits to probabilities
    probs = torch.softmax(outputs.logits, dim=-1)

    # Get predicted label index
    predicted_label_index = torch.argmax(probs).item()

    # Get predicted label in string format
    predicted_label = labels[predicted_label_index]

    return predicted_label


In [4]:
for i,key in labels.items() :
    print(i,"->",key)

0 -> Chinh_tri_Xa_hoi
1 -> Cong_nghe_thong_tin
2 -> Doi_song
3 -> Khoa_hoc
4 -> Kinh_doanh
5 -> Phap_luat
6 -> Suc_khoe
7 -> The_gioi
8 -> The_thao
9 -> Van_hoa


In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("/kaggle/input/vietnews/vietnews_segmented.csv")

In [7]:
df.drop(columns=["Unnamed: 0"], inplace=True)
df.columns

Index(['Document', 'Summary', 'Dataset', 'Document_segmented'], dtype='object')

In [8]:
df.dropna(inplace=True)

In [9]:
empty_segmentation_rows = df[df['Document_segmented'].isna()]

print(empty_segmentation_rows)

Empty DataFrame
Columns: [Document, Summary, Dataset, Document_segmented]
Index: []


In [10]:
from tqdm import tqdm

# Đảm bảo cài đặt thư viện tqdm
# pip install tqdm

# Sử dụng tqdm để theo dõi tiến độ
tqdm.pandas()

# Áp dụng hàm classify_text cho cột segmentation trong DataFrame df
df['category'] = df['Document_segmented'].progress_apply(classify_text)

100%|██████████| 150705/150705 [46:08<00:00, 54.44it/s]


In [11]:
df.to_csv("/kaggle/working/vietnews_category.csv")

In [12]:
df.head()

,Document,Summary,Dataset,Document_segmented,category
0,test,test,vietnews,test,Suc_khoe
1,"Theo lời khai của Huy tại phiên toà , để có ti...","Ngày 25/2 , TAND TP. Đà Nẵng tuyên phạt Hồ Xuâ...",vietnews,"Theo lời khai của Huy tại phiên_toà , để có ti...",Phap_luat
2,"Tối 27/6 báo Người Lao Động đưa tin , một lãnh...",Phát hiện nam thanh niên đang thủ dâm trên xe ...,vietnews,"Tối 27/6 báo Người Lao_Động đưa tin , một lãnh...",Phap_luat
3,Quảng cáo như thần dượcTrên diễn đàn của các “...,“ Baby Support ” và “ Hello baby ” là hai cái ...,vietnews,Quảng_cáo như thần dượcTrên diễn_đàn của các “...,Suc_khoe
4,"Như báo Người Đưa Tin đã phản ánh trước đó , T...",Liên quan đến vụ gây rối tại trạm thu phí trên...,vietnews,"Như báo Người Đưa_Tin đã phản_ánh trước đó , T...",Phap_luat


In [13]:
from collections import Counter

# Lấy cột 'category' từ DataFrame và chuyển thành list
categories = df['category'].tolist()

# Sử dụng Counter để đếm số lượng mẫu thuộc từng category
category_counts = Counter(categories)

# In ra số lượng mẫu của mỗi category
for category, count in category_counts.items():
    print(f"{category}: {count}")

Suc_khoe: 2047
Phap_luat: 29312
The_gioi: 46656
Chinh_tri_Xa_hoi: 35864
Cong_nghe_thong_tin: 3716
Doi_song: 6541
Van_hoa: 6969
Kinh_doanh: 15971
The_thao: 1466
Khoa_hoc: 2163


In [14]:
law_rows = df.loc[df['category'] == 'Phap_luat']
law_rows.head(20)

,Document,Summary,Dataset,Document_segmented,category
1,"Theo lời khai của Huy tại phiên toà , để có ti...","Ngày 25/2 , TAND TP. Đà Nẵng tuyên phạt Hồ Xuâ...",vietnews,"Theo lời khai của Huy tại phiên_toà , để có ti...",Phap_luat
2,"Tối 27/6 báo Người Lao Động đưa tin , một lãnh...",Phát hiện nam thanh niên đang thủ dâm trên xe ...,vietnews,"Tối 27/6 báo Người Lao_Động đưa tin , một lãnh...",Phap_luat
4,"Như báo Người Đưa Tin đã phản ánh trước đó , T...",Liên quan đến vụ gây rối tại trạm thu phí trên...,vietnews,"Như báo Người Đưa_Tin đã phản_ánh trước đó , T...",Phap_luat
5,"Sáng 28/5 , VKSND tỉnh Sóc Trăng tổ chức buổi ...",Bà Huỳnh Ngọc Bích được mọi người biết đến là ...,vietnews,"Sáng 28/5 , VKSND tỉnh Sóc_Trăng tổ_chức buổi ...",Phap_luat
6,"Ngày 7/8 , cơ quan CSĐT Công an tỉnh Vĩnh Long...",Bằng trả phòng và tính tiền thì xảy ra mâu thu...,vietnews,"Ngày 7/8 , cơ_quan CSĐT Công_an tỉnh Vĩnh_Long...",Phap_luat
7,"Ngày 5/9 , cơ quan CSĐT Công an TP. HCM hoàn t...","Cơ quan điều tra xác định , vụ tai nạn chìm tà...",vietnews,"Ngày 5/9 , cơ_quan CSĐT Công_an TP. HCM hoàn_t...",Phap_luat
8,"Ngày 21/8 , TAND Cấp cao tại TP. HCM đã tuyên ...",Bị cáo Tuyết bị cấp sơ thẩm 2 lần tuyên phạt 1...,vietnews,"Ngày 21/8 , TAND Cấp cao tại TP. HCM đã tuyên ...",Phap_luat
9,Nguồn : Công an Hà Tĩnh .Đường dây đánh bạc...,Lực lượng hùng hậu được trang bị bộ đà...,vietnews,Nguồn : Công_an Hà_Tĩnh . Đường dây đánh bạ...,Phap_luat
10,"Chiều 14/5 , ông Nguyễn Ngọc Lâm , trưởng Công...","Khi đi thả lưới , người dân hoảng hốt phát hiệ...",vietnews,"Chiều 14/5 , ông Nguyễn_Ngọc_Lâm , trưởng Công...",Phap_luat
11,"Thông tin trên báo Giao Thông , phòng Cảnh sát...",Nguyễn Văn Tú đã thuê xe chở cả một chiếc máy ...,vietnews,"Thông_tin trên báo Giao_Thông , phòng Cảnh_sát...",Phap_luat


In [15]:
pd.set_option('display.max_colwidth', None)